# Sentiment Analysis Using RNN

# 

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [3]:
df=pd.read_csv(r"Downloads\train.tsv",sep="\t")

In [4]:
d=pd.read_csv(r"Downloads\test.tsv",sep='\t')

In [5]:
d

,PhraseId,SentenceId,Phrase
0,156061,8545,An intermittently pleasing but mostly routine ...
1,156062,8545,An intermittently pleasing but mostly routine ...
2,156063,8545,An
3,156064,8545,intermittently pleasing but mostly routine effort
4,156065,8545,intermittently pleasing but mostly routine
...,...,...,...
66287,222348,11855,"A long-winded , predictable scenario ."
66288,222349,11855,"A long-winded , predictable scenario"
66289,222350,11855,"A long-winded ,"
66290,222351,11855,A long-winded


# Removal of Stopwords 

In [6]:
from nltk.corpus import stopwords
import string
s=set(stopwords.words('english'))
p=list(string.punctuation)
s.update(p)
s

{'!',
 '"',
 '#',
 '$',
 '%',
 '&',
 "'",
 '(',
 ')',
 '*',
 '+',
 ',',
 '-',
 '.',
 '/',
 ':',
 ';',
 '<',
 '=',
 '>',
 '?',
 '@',
 '[',
 '\\',
 ']',
 '^',
 '_',
 '`',
 'a',
 'about',
 'above',
 'after',
 'again',
 'against',
 'ain',
 'all',
 'am',
 'an',
 'and',
 'any',
 'are',
 'aren',
 "aren't",
 'as',
 'at',
 'be',
 'because',
 'been',
 'before',
 'being',
 'below',
 'between',
 'both',
 'but',
 'by',
 'can',
 'couldn',
 "couldn't",
 'd',
 'did',
 'didn',
 "didn't",
 'do',
 'does',
 'doesn',
 "doesn't",
 'doing',
 'don',
 "don't",
 'down',
 'during',
 'each',
 'few',
 'for',
 'from',
 'further',
 'had',
 'hadn',
 "hadn't",
 'has',
 'hasn',
 "hasn't",
 'have',
 'haven',
 "haven't",
 'having',
 'he',
 'her',
 'here',
 'hers',
 'herself',
 'him',
 'himself',
 'his',
 'how',
 'i',
 'if',
 'in',
 'into',
 'is',
 'isn',
 "isn't",
 'it',
 "it's",
 'its',
 'itself',
 'just',
 'll',
 'm',
 'ma',
 'me',
 'mightn',
 "mightn't",
 'more',
 'most',
 'mustn',
 "mustn't",
 'my',
 'myself',
 'need

# Lemmatization

In [7]:
from nltk import pos_tag
from nltk.stem import WordNetLemmatizer
lemmatizer=WordNetLemmatizer()

In [8]:
from nltk.corpus import wordnet
def get_simple_pos(tag):
    if(tag.startswith('J')):
        return wordnet.ADJ
    elif tag.startswith('V'):
        return wordnet.VERB
    elif tag.startswith('N'):
        return wordnet.NOUN
    elif tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN
    

In [9]:
def clean_word(words):
  output=[]
  inputs = words.split()
  for w in inputs:
    if not type(w)==str:
        continue
    if w.lower() not in s:
      #print(w)
      pos=pos_tag([w])
      clean=lemmatizer.lemmatize(w,pos=get_simple_pos(pos[0][1]))
      output.append(w.lower())
  
  return output



In [10]:
x=df.iloc[:,-2].values
x_train_data=[clean_word(i) for i in x]

%matplotlib inline

In [11]:
x_test=d.iloc[:,-1].values
x_test_data=[clean_word(i) for i in x_test]
print(x_train_data[0])

['series', 'escapades', 'demonstrating', 'adage', 'good', 'goose', 'also', 'good', 'gander', 'occasionally', 'amuses', 'none', 'amounts', 'much', 'story']


In [12]:
y=df.iloc[:,-1].values




y_train=y.reshape(-1,1)
#y_test=y_test.reshape(-1,1)

from sklearn.preprocessing import LabelEncoder,OneHotEncoder
o=OneHotEncoder(sparse=False)
y_train=o.fit_transform(y_train)
#y_test=o.fit_transform(y_test)
y_train.shape

(156060, 5)

# Splitting of data

In [19]:
from sklearn.model_selection import train_test_split
x_train,x_val,y_train,y_val=train_test_split(x_train_data,y_train,test_size=0.2,stratify=y_train)

In [20]:
x_train[2342]

['drawing']

# Collection of Unique word 

In [21]:
unique_word=[]
s=set()
max_len=0;
for i in x_train_data:
  #print(i)
  s.update(i)
  if(max_len<len(i)):
    #print(len(i))
    max_len=len(i)
    ss=i

In [22]:
vocab=len(list(s))
oov = '<OOV>'
embed_dim=300
padding='post'
trunc='post'
len(y_train)

124848

# Tokenization and Padding

In [23]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

tokenizer=Tokenizer(num_words=vocab,oov_token=oov,char_level=False)
tokenizer.fit_on_texts(list(x_train))

x_train=tokenizer.texts_to_sequences(x_train)
x_train=pad_sequences(x_train,maxlen=max_len,padding=padding,truncating=trunc)

x_val=tokenizer.texts_to_sequences(x_val)
x_val=pad_sequences(x_val,maxlen=max_len,padding=padding,truncating=trunc)

x_test=tokenizer.texts_to_sequences(x_test)
x_test=pad_sequences(x_test,maxlen=max_len,padding=padding,truncating=trunc)

In [24]:
x_train[3]

array([7074,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0])

# Implementing RNN model with Lstm 

In [30]:
from keras.layers import Embedding,Bidirectional,Dense,LSTM,Dropout
from keras.models import Sequential

model=Sequential()

model.add(Embedding(vocab,embed_dim,input_length = max_len))
model.add(Bidirectional(LSTM(128, dropout = 0.8, recurrent_dropout=0.8, return_sequences=True)))
model.add(Bidirectional(LSTM(64,dropout = 0.5,recurrent_dropout=0.5,return_sequences=False)))
model.add(Dense(128,activation='relu'))
model.add(Dropout(0.4))
model.add(Dense(5,activation='softmax'))
model.compile(loss = 'categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 32, 300)           4915500   
_________________________________________________________________
bidirectional (Bidirectional (None, 32, 256)           439296    
_________________________________________________________________
bidirectional_1 (Bidirection (None, 128)               164352    
_________________________________________________________________
dense (Dense)                (None, 128)               16512     
_________________________________________________________________
dropout (Dropout)            (None, 128)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 5)                 645       
Total params: 5,536,305
Trainable params: 5,536,305
Non-trainable params: 0
______________________________________________

# Early stopping

In [41]:
from keras.callbacks import EarlyStopping
early_stopping = EarlyStopping(min_delta = 0.001,mode = 'max',monitor = 'val_accuracy',patience = 2)
callback = [early_stopping]

# Training Model

In [42]:
num_epochs = 4

history = model.fit(x_train,y_train,validation_data = (x_val, y_val),epochs = num_epochs,batch_size = 256,verbose = 1,callbacks = callback)

Epoch 1/4
488/488 [==============================] - 1801s 4s/step - loss: 0.7382 - accuracy: 0.6982 - val_loss: 0.8404 - val_accuracy: 0.6624
Epoch 2/4
488/488 [==============================] - 1524s 3s/step - loss: 0.7113 - accuracy: 0.7068 - val_loss: 0.8414 - val_accuracy: 0.6625
Epoch 3/4
488/488 [==============================] - 1623s 3s/step - loss: 0.6870 - accuracy: 0.7144 - val_loss: 0.8570 - val_accuracy: 0.6617


In [43]:
pred = np.argmax(model.predict(x_test),axis=-1)


In [44]:
x_test.shape

(66292, 32)

In [45]:
submission = pd.DataFrame({'PhraseID':d['PhraseId'],'Sentiment':pred})
submission.to_csv('Submission.csv', index=False)
submission

,PhraseID,Sentiment
0,156061,2
1,156062,2
2,156063,2
3,156064,2
4,156065,2
...,...,...
66287,222348,1
66288,222349,1
66289,222350,1
66290,222351,1
